In [35]:
import os
import numpy as np
import tensorflow as tf 
import tensorflow.keras.backend as K
from  tensorflow.keras import models, activations, layers

## Constants 

In [36]:
batch_size = 128
num_classes = 10
epochs = 10

## Load data 

In [37]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [38]:
print("Loaded data;\nx_train shape \t{}\ny_train shape \t{}\nx_test shape \t{}\ny_test shape \t{}".format(
    x_train.shape, y_train.shape, 
    x_test.shape, y_test.shape))

Loaded data;
x_train shape 	(60000, 28, 28)
y_train shape 	(60000,)
x_test shape 	(10000, 28, 28)
y_test shape 	(10000,)


## Export Training and test data 

In [39]:
x_train.tofile(os.path.join('exports', "x_train.data"))
y_train.tofile(os.path.join('exports', "y_train.data"))

In [40]:
x_test.tofile(os.path.join('exports', "x_test.data"))
y_test.tofile(os.path.join('exports', "y_test.data"))

In [41]:
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
print("Loaded data;\nx_train shape \t{}\ny_train shape \t{}\nx_test shape \t{}\ny_test shape \t{}".format(
    x_train.shape, y_train.shape, 
    x_test.shape, y_test.shape))

Loaded data;
x_train shape 	(60000, 28, 28, 1)
y_train shape 	(60000,)
x_test shape 	(10000, 28, 28, 1)
y_test shape 	(10000,)


## Prepare data 

In [42]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [43]:
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## Shallow network 

In [44]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=input_shape))
model.add(layers.Dense(num_classes, name="fc_1"))
model.add(layers.Activation(activations.softmax, name="output"))

In [45]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
fc_1 (Dense)                 (None, 10)                7850      
_________________________________________________________________
output (Activation)          (None, 10)                0         
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

### Train

In [47]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 19us/step - loss: 0.6486 - acc: 0.8416 - val_loss: 0.3634 - val_acc: 0.9024
Epoch 2/10
60000/60000 [==============================] - 1s 14us/step - loss: 0.3487 - acc: 0.9042 - val_loss: 0.3096 - val_acc: 0.9160
Epoch 3/10
60000/60000 [==============================] - 1s 14us/step - loss: 0.3149 - acc: 0.9122 - val_loss: 0.2926 - val_acc: 0.9194
Epoch 4/10
60000/60000 [==============================] - 1s 14us/step - loss: 0.3001 - acc: 0.9164 - val_loss: 0.2849 - val_acc: 0.9197
Epoch 5/10
60000/60000 [==============================] - 1s 14us/step - loss: 0.2910 - acc: 0.9191 - val_loss: 0.2799 - val_acc: 0.9231
Epoch 6/10
60000/60000 [==============================] - 1s 14us/step - loss: 0.2849 - acc: 0.9209 - val_loss: 0.2756 - val_acc: 0.9235
Epoch 7/10
60000/60000 [==============================] - 1s 14us/step - loss: 0.2798 - acc: 0.9221 - val_loss: 0.2741 - val_acc

### Evaluate 

In [48]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 0.2709703020751476)
('Test accuracy:', 0.9245)


### Export weights 

The type of each entry in array is given by -dataType. The number of entries is equal to:

*inputFeatureChannels * outputFeatureChannels * kernelHeight * kernelWidth*

The layout of filter weight is as a 4D tensor (array)
weight[ outputChannels ][ kernelHeight ][ kernelWidth ][ inputChannels / groups ]

*Note: For binary-convolutions the layout of the weights are:
weight[ outputChannels ][ kernelHeight ][ kernelWidth ][ floor((inputChannels/groups)+31) / 32 ]
with each 32 sub input feature channel index specified in machine byte order, so that for example
the 13th feature channel bit can be extracted using bitmask = (1U << 13).*

In [49]:
def export_dense_weights(name, wts_coef, bias_coef, kernel_width, kernel_height):
    """
    A fully connected layer in a Convolutional Neural Network (CNN) is one where every input channel is connected 
    to every output channel. The kernel width is equal to the width of the source image, and the 
    kernel height is equal to the height of the source image. The width and height of the output is 1 x 1.
    
    A fully connected layer takes an MPSImage object with dimensions 
    source.width x source.height x Ni, convolves it with Weights[No][source.width][source.height][Ni], 
    and produces a 1 x 1 x No output.
    
    Thus, the following conditions must be true:
    - kernelWidth == source.width
    - kernelHeight == source.height
    - clipRect.size.width == 1
    - clipRect.size.height == 1
    
    You can think of a fully connected layer as a matrix multiplication where the image is 
    flattened into a vector of length source.width*source.height*Ni, and the weights are arranged in a 
    matrix of dimension No x (source.width*source.height*Ni) to produce an output vector of length No.
    
    The value of the strideInPixelsX, strideInPixelsY, and groups properties must be 1. 
    The offset property is not applicable and it is ignored. Because the clip rectangle is 
    clamped to the destination image bounds, if the destination is 1 x 1, you do not need to set the 
    clipRect property.
    """
    print("Exporting weights for {}\n\t{}\n\t{}".format(name, 
          os.path.join('exports', "{}_conv_wts.data".format(name)), 
          os.path.join('exports', "{}_bias_wts.data".format(name))))
        
    input_feature_channels = int(wts_coef.shape[0] / kernel_width / kernel_height) 
    output_feature_channels = wts_coef.shape[-1]            
    
    print("\tOriginal weights shape {}".format(wts_coef.shape))    
    wts_coef = np.reshape(wts_coef, [kernel_width, kernel_height, -1, output_feature_channels])    
        
    if bias_coef is not None:
        # [output_feature_channels]
        print("\tOriginal bias shape {}".format(bias_coef.shape))
    
    # [output_feature_channels, kernel_width, kernel_height, input_feature_channels]
    wts_coef = wts_coef.transpose(3, 0, 1, 2)
    print("\tReshaped weights shape {}".format(wts_coef.shape))    
    wts_coef.tofile(os.path.join('exports', "shallow_{}_wts.data".format(name)))    
    
    if bias_coef is not None:
        bias_coef = np.squeeze(bias_coef)
        print("\tReshaped bias_coef shape {}".format(bias_coef.shape))    
        bias_coef.tofile(os.path.join('exports', "shallow_{}_bias_terms.data".format(name)))   
        
    print("\n")

In [50]:
flatted_input_kernel_width = None
flatted_input_kernel_height = None

for layer in model.layers:        
    if "flatten" in layer.name:
        flatted_input_kernel_width = layer.input_shape[1] # None, 14, 14, 64
        flatted_input_kernel_height = layer.input_shape[2] # None, 14, 14, 64
        
    if len(layer.get_weights()) > 0:        
        name = layer.name         
        wts = layer.get_weights()
        
        export_dense_weights(layer.name, wts[0], wts[1] if len(wts) == 2 else None, 
                                flatted_input_kernel_width, flatted_input_kernel_height)        
        # after the initial pass (from cnn to fcn); flattern the kernel down to 1x1 
        # i.e. update the flatted_input_kernel_DIM to have the kernel width and height of 1 
        flatted_input_kernel_width, flatted_input_kernel_height = 1, 1 

Exporting weights for fc_1
	exports/fc_1_conv_wts.data
	exports/fc_1_bias_wts.data
	Original weights shape (784, 10)
	Original bias shape (10,)
	Reshaped weights shape (10, 28, 28, 1)
	Reshaped bias_coef shape (10,)




## Deeper network (1 hidden layer)

In [51]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=input_shape))
model.add(layers.Dense(32, name="fc_1"))
model.add(layers.Activation(activations.relu))
model.add(layers.Dense(num_classes, name="fc_2"))
model.add(layers.Activation(activations.softmax, name="output"))

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
fc_1 (Dense)                 (None, 32)                25120     
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
fc_2 (Dense)                 (None, 10)                330       
_________________________________________________________________
output (Activation)          (None, 10)                0         
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

### Train

In [54]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 23us/step - loss: 0.4499 - acc: 0.8766 - val_loss: 0.2708 - val_acc: 0.9238
Epoch 2/10
60000/60000 [==============================] - 1s 17us/step - loss: 0.2478 - acc: 0.9299 - val_loss: 0.2195 - val_acc: 0.9362
Epoch 3/10
60000/60000 [==============================] - 1s 19us/step - loss: 0.2080 - acc: 0.9408 - val_loss: 0.1938 - val_acc: 0.9416
Epoch 4/10
60000/60000 [==============================] - 1s 18us/step - loss: 0.1827 - acc: 0.9478 - val_loss: 0.1702 - val_acc: 0.9494
Epoch 5/10
60000/60000 [==============================] - 1s 17us/step - loss: 0.1627 - acc: 0.9540 - val_loss: 0.1590 - val_acc: 0.9534
Epoch 6/10
60000/60000 [==============================] - 1s 18us/step - loss: 0.1478 - acc: 0.9580 - val_loss: 0.1510 - val_acc: 0.9553
Epoch 7/10
60000/60000 [==============================] - 1s 19us/step - loss: 0.1358 - acc: 0.9609 - val_loss: 0.1407 - val_acc

### Export weights

In [55]:
def export_dense_weights(name, wts_coef, bias_coef, kernel_width, kernel_height):
    """
    A fully connected layer in a Convolutional Neural Network (CNN) is one where every input channel is connected 
    to every output channel. The kernel width is equal to the width of the source image, and the 
    kernel height is equal to the height of the source image. The width and height of the output is 1 x 1.
    
    A fully connected layer takes an MPSImage object with dimensions 
    source.width x source.height x Ni, convolves it with Weights[No][source.width][source.height][Ni], 
    and produces a 1 x 1 x No output.
    
    Thus, the following conditions must be true:
    - kernelWidth == source.width
    - kernelHeight == source.height
    - clipRect.size.width == 1
    - clipRect.size.height == 1
    
    You can think of a fully connected layer as a matrix multiplication where the image is 
    flattened into a vector of length source.width*source.height*Ni, and the weights are arranged in a 
    matrix of dimension No x (source.width*source.height*Ni) to produce an output vector of length No.
    
    The value of the strideInPixelsX, strideInPixelsY, and groups properties must be 1. 
    The offset property is not applicable and it is ignored. Because the clip rectangle is 
    clamped to the destination image bounds, if the destination is 1 x 1, you do not need to set the 
    clipRect property.
    """
    print("Exporting weights for {}\n\t{}\n\t{}".format(name, 
          os.path.join('exports', "{}_conv_wts.data".format(name)), 
          os.path.join('exports', "{}_bias_wts.data".format(name))))
        
    input_feature_channels = int(wts_coef.shape[0] / kernel_width / kernel_height) 
    output_feature_channels = wts_coef.shape[-1]            
    
    print("\tOriginal weights shape {}".format(wts_coef.shape))    
    wts_coef = np.reshape(wts_coef, [kernel_width, kernel_height, -1, output_feature_channels])    
        
    if bias_coef is not None:
        # [output_feature_channels]
        print("\tOriginal bias shape {}".format(bias_coef.shape))
    
    # [output_feature_channels, kernel_width, kernel_height, input_feature_channels]
    wts_coef = wts_coef.transpose(3, 0, 1, 2)
    print("\tReshaped weights shape {}".format(wts_coef.shape))    
    wts_coef.tofile(os.path.join('exports', "h1_{}_wts.data".format(name)))    
    
    if bias_coef is not None:
        bias_coef = np.squeeze(bias_coef)
        print("\tReshaped bias_coef shape {}".format(bias_coef.shape))    
        bias_coef.tofile(os.path.join('exports', "h1_{}_bias_terms.data".format(name)))   
        
    print("\n")

In [56]:
flatted_input_kernel_width = None
flatted_input_kernel_height = None

for layer in model.layers:        
    if "flatten" in layer.name:
        flatted_input_kernel_width = layer.input_shape[1] # None, 14, 14, 64
        flatted_input_kernel_height = layer.input_shape[2] # None, 14, 14, 64
        
    if len(layer.get_weights()) > 0:        
        name = layer.name         
        wts = layer.get_weights()
        
        export_dense_weights(layer.name, wts[0], wts[1] if len(wts) == 2 else None, 
                                flatted_input_kernel_width, flatted_input_kernel_height)        
        # after the initial pass (from cnn to fcn); flattern the kernel down to 1x1 
        # i.e. update the flatted_input_kernel_DIM to have the kernel width and height of 1 
        flatted_input_kernel_width, flatted_input_kernel_height = 1, 1 

Exporting weights for fc_1
	exports/fc_1_conv_wts.data
	exports/fc_1_bias_wts.data
	Original weights shape (784, 32)
	Original bias shape (32,)
	Reshaped weights shape (32, 28, 28, 1)
	Reshaped bias_coef shape (32,)


Exporting weights for fc_2
	exports/fc_2_conv_wts.data
	exports/fc_2_bias_wts.data
	Original weights shape (32, 10)
	Original bias shape (10,)
	Reshaped weights shape (10, 1, 1, 32)
	Reshaped bias_coef shape (10,)


